# Kaynakları Yükle

* Python 3.10.10 enviromentını kurduktan sonra, torch2.0.1+cu118 PyTorch paketini de kurmamız gerekmektedir.

Pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118 
* Gerekli kaynakları, pip install -r requirements.txt kodu ile python kütüphanesine yüklediğinizden emin olunuz.
* Bu proje içerisinde örnek model bulunmamaktadır. 
* En doğru sonuçların alınabilmesi için, VolumeCalculator-Training ile eğitilen model kullanılmalıdır.
 -------------------------------------------------------------
* After installing Python 3.10.10 environment, it is necessary to install the torch2.0.1+cu118 PyTorch package as well.
    
pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
* Make sure you have installed the necessary dependencies into the Python library using the following command:
    
pip install -r requirements.txt
* There is no sample model in project.
* To obtain the most accurate results, the model trained with VolumeCalculator-Training should be used.

In [1]:
import cv2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import Metadata
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import yaml
import numpy
from aioflask import Flask, jsonify, request
import time
from PIL import Image
import asyncio
import base64

Get Time

In [2]:
def getTime():
    # Get the current timestamp in seconds
    current_timestamp = time.time()

    # Extract the current minute, second, and millisecond
    current_minute = int((current_timestamp // 60) % 60)
    current_second = int(current_timestamp % 60)
    current_millisecond = int((current_timestamp % 1) * 1000)

    # Format the time as string (mm:ss.sss)
    currentTime = f"{current_minute:02}:{current_second:02}.{current_millisecond:03}"
    return currentTime

Volume Calculator

In [4]:
def calculateVolume(x, y, x2, y2, dpi, pixelcount):
    
    # Define the data points for the volume calculation
    volx = numpy.array([148020, 174503, 374399, 477370, 662724])
    voly = numpy.array([250, 330, 1000, 1500, 2500])

    # Set the degree of the polynomial equation
    polynomial_degree = 2

    # Calculate coefficients of the polynomial equation
    coefficients = numpy.polyfit(volx, voly, polynomial_degree)

    # Create the polynomial equation
    poly_eq = numpy.poly1d(coefficients)

    # Specify the pixel count for prediction
    new_x = pixelcount

    # Predict the corresponding y value using the polynomial equation
    predicted_y = poly_eq(new_x)
    
    # Round the predicted volume
    volume = round(predicted_y)

    # Calculate the difference in pixels
    x = x2 - x
    y = y2 - y

    # Convert pixel differences to centimeters using DPI (dots per inch) value
    x = x * 2.54 / dpi
    y = y * 2.54 / dpi

    # Calculate the diameter of the mask in centimeters
    diameter = y / 2
    diameter = diameter * diameter
    
    # Adjust dimensions based on specific conditions
    if not (x <= 0 and y <= 0):
        x = round(x * 10, 1)  # Convert to millimeters and round to one decimal place
        y = round(y * 10, 1)  # Convert to millimeters and round to one decimal place
        x = float(x) * 1.554   # Apply a scaling factor
        x = float(x) - 77      # Adjust by a constant value
        y = float(y) * 1.134   # Apply a scaling factor
        y = float(y) - 2.7     # Adjust by a constant value
    else:
        x = 0
        y = 0

    # Calculate the mask dimensions in millimeters
    mask_dimensions_mm = {"x": round(x * 1), "y": round(y * 1)}

    return volume, mask_dimensions_mm


In [5]:
def draw_and_show_image(image, instance, filename):
    
    # Create a Visualizer object with the input image and instance predictions
    image_visualizer = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1)
    image_with_bb = image_visualizer.draw_instance_predictions(instance.to("cpu"))

    # Extract the output image from the Visualizer object and perform BGR to RGB conversion
    output_image = image_with_bb.get_image()[:, :, ::-1]

    # Display the output image
    cv2.imshow("image", output_image)
    cv2.waitKey()



We specify the path to the metadata.yaml file because we will retrieve class names from the Metadata file. Then, we create a dictionary.

In [6]:
# Find metadata file and load
with open("./model/metadata.yaml", "r") as f:
    metadata_dict = yaml.load(f, Loader=yaml.FullLoader)

We can continue to create dictionary if yaml loaded successfully

In [7]:
# Setup metadata 
metadata = Metadata()
metadata.set(thing_classes=metadata_dict["thing_classes"])
metadata.set(thing_colors=metadata_dict["thing_colors"])
metadata.set(thing_dataset_id_to_contiguous_id=metadata_dict["thing_dataset_id_to_contiguous_id"])

namespace(thing_classes=['bottle'],
          thing_colors=[[220, 20, 60],
                        [119, 11, 32],
                        [0, 0, 142],
                        [0, 0, 230],
                        [106, 0, 228],
                        [0, 60, 100],
                        [0, 80, 100],
                        [0, 0, 70],
                        [0, 0, 192],
                        [250, 170, 30],
                        [100, 170, 30],
                        [220, 220, 0],
                        [175, 116, 175],
                        [250, 0, 30],
                        [165, 42, 42],
                        [255, 77, 255],
                        [0, 226, 252],
                        [182, 182, 255],
                        [0, 82, 0],
                        [120, 166, 157],
                        [110, 76, 0],
                        [174, 57, 255],
                        [199, 100, 0],
                        [72, 0, 118],
                        [255, 179, 

We specify the path of the configuration file, which contains all the settings of the trained model

In [8]:
# Load a configuration file
cfg = get_cfg()
cfg.merge_from_file("./model/config.yaml")

Set model path

In [9]:
# Replace with your .pth file path
model_path = "./model/model_final.pth"
cfg.MODEL.WEIGHTS = model_path

We set the device on which the model will run as CUDA and specify our threshold.

In [10]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.10  # set threshold for this model
cfg.MODEL.DEVICE = "cuda"

We make the necessary definitions for prediction.

In [11]:
# Create a predictor
predictor = DefaultPredictor(cfg)
loop = asyncio.get_event_loop()
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor()

Load Image

In [12]:

image = Image.open("./images.jpg")
image = numpy.asarray(image)

Start predict

In [13]:
# Use the predictor to make a prediction
print("Predict started at ",getTime())
outputs = predictor(image)
print("Predict finished at ",getTime())

Predict started at  33:37.679


f:\Users\mbaru\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Predict finished at  33:39.391


Get outputs

In [14]:
instances = outputs["instances"]

View results

In [15]:
results_list = []
response = dict()
for i in range(len(instances)):
    prediction_dict = dict()

    pred_boxes = instances.get("pred_boxes")
    if pred_boxes is None and len(pred_boxes.tensor) == 0:
        print("No object detected!")
        response["message"] = "No object detected!"
        print(response)
    elif i < len(instances):
        pred = instances[i]
        y1, x1, y2, x2 = pred.pred_boxes.tensor[0].tolist()
        y1, x1, y2, x2 = int(y1), int(x1), int(y2), int(x2)  # convert to integers
        print(y1, ' ', x1,' ',y2,' ',x2)
        mask = pred.pred_masks[0][y1:y2, x1:x2].detach().cpu().numpy()
        mask = mask.astype(int)
        num_pixels = (pred.pred_masks[0].detach().cpu().numpy() == True).sum()
        print(num_pixels)

        # Toplam hacmi hesapla
        volume, mask_dimensions_mm = calculateVolume(y1, x1, y2, x2, 135,num_pixels)
        print("Volume -> ", volume," ", num_pixels)
        print("Mask Dimensions (mm) -> ", mask_dimensions_mm)

        # Retrieve the prediction score and threshold
        scores = instances[i].scores.item()
        threshold = cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST

        # Retrieve the class tag (name)
        class_label = instances[i].pred_classes.item()
        class_tag = metadata.thing_classes[class_label]
        prediction_dict["predictions"] = {
            "class_tag": class_tag,
            "score": round(scores,2),
            "threshold": threshold,
            "mask_dimensions": mask_dimensions_mm,#{"x": round(x_dim,2), "y": round(y_dim,2)},
            "volume": round(volume,1),
            "num_pixels": int(num_pixels)
            #"prediction_image_base64": prediction_base64
        }
        results_list.append(prediction_dict)
        
    else:
        print("Index out of range for instances!")
        
        response["message"] = "Index out of range for instances!"
        print(response)


pred_boxes = instances.get("pred_boxes")
if pred_boxes is not None and len(pred_boxes.tensor) > 0:
    draw_and_show_image(image,instances[0],"filename")
response["results"] = results_list
response["message"] = "OK"

print(response)








167   238   896   476
129496
Volume ->  205   129496
Mask Dimensions (mm) ->  {'x': 136, 'y': 48}
113   253   460   373
38000
Volume ->  8   38000
Mask Dimensions (mm) ->  {'x': 24, 'y': 23}
119   236   308   509
39758
Volume ->  11   39758
Mask Dimensions (mm) ->  {'x': -22, 'y': 56}
{'results': [{'predictions': {'class_tag': 'bottle', 'score': 0.99, 'threshold': 0.1, 'mask_dimensions': {'x': 136, 'y': 48}, 'volume': 205, 'num_pixels': 129496}}, {'predictions': {'class_tag': 'bottle', 'score': 0.64, 'threshold': 0.1, 'mask_dimensions': {'x': 24, 'y': 23}, 'volume': 8, 'num_pixels': 38000}}, {'predictions': {'class_tag': 'bottle', 'score': 0.11, 'threshold': 0.1, 'mask_dimensions': {'x': -22, 'y': 56}, 'volume': 11, 'num_pixels': 39758}}], 'message': 'OK'}
